In [6]:
from torchvision import models

In [7]:
print("hi")

hi


In [8]:
vgg = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/hang/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [01:33<00:00, 5.89MB/s]


In [9]:
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [34]:
vgg_dict = { k: v for k,v in  vgg.state_dict().items()}

In [35]:
for k in vgg_dict:
    print(k)

features.0.weight
features.0.bias
features.2.weight
features.2.bias
features.5.weight
features.5.bias
features.7.weight
features.7.bias
features.10.weight
features.10.bias
features.12.weight
features.12.bias
features.14.weight
features.14.bias
features.17.weight
features.17.bias
features.19.weight
features.19.bias
features.21.weight
features.21.bias
features.24.weight
features.24.bias
features.26.weight
features.26.bias
features.28.weight
features.28.bias
classifier.0.weight
classifier.0.bias
classifier.3.weight
classifier.3.bias
classifier.6.weight
classifier.6.bias


In [ ]:
vgg_dict

In [14]:
import torch.nn as nn
import torch
from torchvision import models
from utils import save_net,load_net

class CSRNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CSRNet, self).__init__()
        self.seen = 0
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat  = [512, 512, 512,256,128,64]
        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat,in_channels = 512,dilation = True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        # if not load_weights:
        #     mod = models.vgg16(pretrained = True)
        #     self._initialize_weights()
        #     for i in range(len(self.frontend.state_dict().items())):
        #         self.frontend.state_dict().items()[i][1].data[:] = mod.state_dict().items()[i][1].data[:]
    
    def forward(self,x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            
                
def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)                

In [15]:
model = CSRNet()

In [23]:
tem = {k:v for k, v in model.frontend.state_dict().items()}

In [25]:
for k in tem:
    print(k)

0.weight
0.bias
2.weight
2.bias
5.weight
5.bias
7.weight
7.bias
10.weight
10.bias
12.weight
12.bias
14.weight
14.bias
17.weight
17.bias
19.weight
19.bias
21.weight
21.bias


In [31]:
tem['0.weight']

tensor([[[[ 0.0914, -0.1180,  0.1049],
          [ 0.1479, -0.0080, -0.0952],
          [-0.1904, -0.1134, -0.1803]],

         [[ 0.1744,  0.1923,  0.1741],
          [ 0.1563, -0.0596, -0.0509],
          [-0.1682,  0.1444,  0.1384]],

         [[-0.0643, -0.1868,  0.0416],
          [-0.1189, -0.0877, -0.1616],
          [ 0.0969, -0.0021,  0.0601]]],


        [[[ 0.1483,  0.1916,  0.0682],
          [ 0.1739, -0.0262,  0.0132],
          [ 0.0548,  0.1791, -0.1157]],

         [[ 0.0254,  0.1337,  0.0600],
          [ 0.0471,  0.0198, -0.1535],
          [ 0.1668,  0.1908, -0.1569]],

         [[ 0.0415,  0.1057, -0.0806],
          [-0.1021,  0.1550, -0.1712],
          [-0.0975,  0.0737, -0.0408]]],


        [[[-0.1321,  0.0401, -0.0989],
          [ 0.0663,  0.0818,  0.0561],
          [ 0.1557, -0.1012,  0.0614]],

         [[-0.0418, -0.0063, -0.0104],
          [-0.1732,  0.0542,  0.0457],
          [ 0.0782,  0.1920, -0.1191]],

         [[-0.0204,  0.1866,  0.0210],
     

In [30]:
torch.sum(tem['0.weight'])

tensor(-2.4552)

In [28]:
tem['0.bias'].shape

torch.Size([64])

In [29]:
tem['2.weight'].shape

torch.Size([64, 64, 3, 3])